# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [2]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [3]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [4]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7ff7d0cc20a0>)

In [5]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [6]:
df

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-02,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-03,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-04,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-05,0.0,NaN,0.0,NaN,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88240,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-11-24,NaN,NaN,NaN,NaN,...,NaN,67.59,NaN,73.81,NaN,58.33,NaN,63.89,NaN,25.0
88241,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-11-25,NaN,NaN,NaN,NaN,...,NaN,67.59,NaN,73.81,NaN,58.33,NaN,63.89,NaN,25.0
88242,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-11-26,NaN,NaN,NaN,NaN,...,NaN,67.59,NaN,73.81,NaN,58.33,NaN,63.89,NaN,25.0
88243,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2020-11-27,NaN,NaN,NaN,NaN,...,NaN,67.59,NaN,73.81,NaN,58.33,NaN,63.89,NaN,25.0


In [7]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [8]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

<ipython-input-8-9010e734381b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)


In [9]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

<ipython-input-9-a3c4998d92a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)


In [10]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [11]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88120,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
88121,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
88122,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
88123,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [12]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [13]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [15]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols                     # sceglie le colonne "NewCases" + tutti i NPIs
y_col = cases_col                                 # L'output è solo il numero di Nuovi Casi
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()

for g in geo_ids:
    
    gdf = df[df.GeoID == g]                   # Dataframe per il GeoID
    all_case_data = np.array(gdf[cases_col])  # nuovi casi in ordine cronologico shape = (len(gdf), 1)
    all_npi_data = np.array(gdf[npi_cols])    # npi in ordine cronologico shape = (len(gdf), 12)

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)  # questo è il numero totale di giorni che abbiamo a disposizione
    
    for d in range(nb_lookback_days, nb_total_days - 1): # questo for va da 30 alla fine
        X_cases = all_case_data[d - nb_lookback_days: d] # prende i range 0-30, 1-31, 2-32 etc...
        # X_cases è (30 x 1)
        
        
        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d] # Higher NPI means LOWER NewCases, stessi range sopra
        # X_npis è (30 x 12)

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(), X_npis.flatten()]) # 1 sample è (30 NC + 30 x 12 PIs,)
        # X_sample è un vettore con in fila tutti i NC e NPI, 
        # come se lo spazio fosse composto da 390 variabili
        
        y_sample = all_case_data[d + 1] # L'output è semplicemente il giorno dopo
        
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [19]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [23]:
# Split data into train and test sets
# Scelta Randomica di test e train set.
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=42)

In [24]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [25]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 259.84015150381117
Test MAE: 254.17028978654804


In [26]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Questo rientra nel contesto dell'explainability 

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -6 NewCases 0.5247728165951634
Day -5 NewCases 0.1210331714268772
Day -3 NewCases 0.09711381862030334
Day -2 NewCases 0.07841775054352185
Day -1 NewCases 0.21972407921421103
Day -26 C4_Restrictions on gatherings 0.15560632624736404
Day -23 C6_Stay at home requirements 20.411046334500515
Day -22 C7_Restrictions on internal movement 2.4120275597048355
Day -17 C6_Stay at home requirements 5.475021870227211
Day -14 C6_Stay at home requirements 9.073259701124135
Day -10 C6_Stay at home requirements 1.2387309022515984
Intercept 44.05469448823396


In [27]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [16]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [17]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../covid_xprize/validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 133.51810815377561
2020-08-02: 158.51751227595548
2020-08-03: 179.30764030240323
2020-08-04: 202.2792297129373
2020-08-05: 204.52205696851195
2020-08-06: 225.03689663720843
2020-08-07: 296.1225300942008
2020-08-08: 329.2035953935809
2020-08-09: 357.72937025706744
2020-08-10: 385.82103208556987
2020-08-11: 399.86422226719947
2020-08-12: 427.29233100993315
2020-08-13: 478.4559852201127
2020-08-14: 514.0251192563574
2020-08-15: 537.8126159725089
2020-08-16: 567.2386834453145
2020-08-17: 589.7131108934607
2020-08-18: 614.3799077698756
2020-08-19: 655.5742088497011
2020-08-20: 690.2872717382089
2020-08-21: 719.5849807021218
2020-08-22: 750.9072005558282
2020-08-23: 775.8254666867851
2020-08-24: 784.1213352284149
2020-08-25: 816.7590735567652
2020-08-26: 848.7632883019012
2020-08-27: 877.859256292197
2020-08-28: 909.3846013070637
2020-08-29: 935.7816745594394
2020-08-30: 955.8056349566982
2020-08-31: 986.3378964398028

Predicting for Afghanistan__nan
20

2020-08-24: 5247.666874435325
2020-08-25: 5297.246959053889
2020-08-26: 5357.229301752524
2020-08-27: 5455.269162440245
2020-08-28: 5546.096433738701
2020-08-29: 5643.884083384859
2020-08-30: 5742.075524410449
2020-08-31: 5813.4175528621

Predicting for Burundi__nan
2020-08-01: 125.44606044719147
2020-08-02: 151.83250577591548
2020-08-03: 169.66309695316394
2020-08-04: 183.59379616619927
2020-08-05: 192.23038812093597
2020-08-06: 210.03610950155473
2020-08-07: 285.00289837919456
2020-08-08: 319.71490631273014
2020-08-09: 345.9929650990085
2020-08-10: 370.125002463733
2020-08-11: 385.06894386394106
2020-08-12: 411.214257118737
2020-08-13: 464.0162739250537
2020-08-14: 500.5161032825843
2020-08-15: 513.7799498078782
2020-08-16: 539.1569261081926
2020-08-17: 560.3242420757847
2020-08-18: 577.4144207577635
2020-08-19: 617.4206460940884
2020-08-20: 650.3662730386014
2020-08-21: 672.4340733172579
2020-08-22: 699.6306743919355
2020-08-23: 718.8888608443381
2020-08-24: 700.3845816255734
2020-0

2020-08-13: 2251.6190945173535
2020-08-14: 2293.1716260153794
2020-08-15: 2315.132576919909
2020-08-16: 2357.019482691742
2020-08-17: 2402.698492635403
2020-08-18: 2445.935946880911
2020-08-19: 2504.050387206253
2020-08-20: 2549.109818609777
2020-08-21: 2584.360833766545
2020-08-22: 2628.793290043104
2020-08-23: 2670.0765831468843
2020-08-24: 2674.956693671583
2020-08-25: 2719.531884362851
2020-08-26: 2761.630499513812
2020-08-27: 2798.2597479468027
2020-08-28: 2842.25181463573
2020-08-29: 2881.133676777949
2020-08-30: 2904.639949276971
2020-08-31: 2945.6132863825974

Predicting for Belize__nan
2020-08-01: 199.61887392022737
2020-08-02: 225.22716745868598
2020-08-03: 249.65675749874353
2020-08-04: 270.95990594373416
2020-08-05: 279.7217811295276
2020-08-06: 290.64063255617395
2020-08-07: 365.86605266832265
2020-08-08: 400.4973468485093
2020-08-09: 430.4644280513837
2020-08-10: 459.309844121057
2020-08-11: 476.0418111605606
2020-08-12: 499.72512512718754
2020-08-13: 552.7100821599212
20

2020-08-24: 7947.476513865948
2020-08-25: 8070.609684642011
2020-08-26: 8205.591283723785
2020-08-27: 8260.55201117233
2020-08-28: 8345.423817881026
2020-08-29: 8456.888936078702
2020-08-30: 8547.955601550644
2020-08-31: 8665.902401709976

Predicting for Switzerland__nan
2020-08-01: 3060.5638667715857
2020-08-02: 7159.810107651998
2020-08-03: 5155.5600925819035
2020-08-04: 5205.443271794007
2020-08-05: 5229.629257227523
2020-08-06: 4804.021437477777
2020-08-07: 4565.592354219822
2020-08-08: 6390.769589094763
2020-08-09: 5686.037097258264
2020-08-10: 5680.266576730932
2020-08-11: 5759.687070299462
2020-08-12: 5456.040648050168
2020-08-13: 5490.775684257189
2020-08-14: 6354.816755954178
2020-08-15: 6129.233566175817
2020-08-16: 6157.38121864377
2020-08-17: 6234.7129639068335
2020-08-18: 6065.913300640985
2020-08-19: 6160.427291519178
2020-08-20: 6601.589600303872
2020-08-21: 6574.569250833503
2020-08-22: 6636.536629120101
2020-08-23: 6706.20361974216
2020-08-24: 6605.782188861873
2020-08

2020-08-09: 407.2292383805601
2020-08-10: 438.79713584695827
2020-08-11: 426.0363425197605
2020-08-12: 449.5593279771632
2020-08-13: 507.4428583643393
2020-08-14: 545.1182133554242
2020-08-15: 581.4680869996369
2020-08-16: 613.0522319572324
2020-08-17: 622.6518461156238
2020-08-18: 644.6429738528832
2020-08-19: 688.2306621938652
2020-08-20: 724.6352194799572
2020-08-21: 761.0860229764445
2020-08-22: 793.9192450180727
2020-08-23: 810.4024874617479
2020-08-24: 816.5436927284322
2020-08-25: 849.6539947673837
2020-08-26: 882.527333927146
2020-08-27: 916.2011355409281
2020-08-28: 948.6184032876313
2020-08-29: 970.9675839789211
2020-08-30: 988.9207002529104
2020-08-31: 1019.1203202848535

Predicting for Costa Rica__nan
2020-08-01: 991.7892213453046
2020-08-02: 2182.373208746918
2020-08-03: 1440.8820675539298
2020-08-04: 1551.6417897350407
2020-08-05: 1592.3922726915512
2020-08-06: 1562.2419006960758
2020-08-07: 1525.1651025280553
2020-08-08: 2053.6526359092923
2020-08-09: 1788.2297814884118


...

2020-08-25: 3551.166577548121
2020-08-26: 3679.723835532931
2020-08-27: 3683.7918646269877
2020-08-28: 3778.253978355659
2020-08-29: 3886.4727900995704
2020-08-30: 3837.7913678212863
2020-08-31: 3907.155838863653

Predicting for United States__Louisiana
2020-08-01: 1411.498061302116
2020-08-02: 2765.2690045963377
2020-08-03: 1488.1188033760018
2020-08-04: 1403.4480246951287
2020-08-05: 3420.3260343831034
2020-08-06: 1298.6798981999589
2020-08-07: 1887.0064924064798
2020-08-08: 2601.612671054048
2020-08-09: 1918.2612645346157
2020-08-10: 2080.961176052711
2020-08-11: 2932.8959814501086
2020-08-12: 2024.3070834733257
2020-08-13: 2302.5043012693795
2020-08-14: 2667.4137826042293
2020-08-15: 2333.186829232873
2020-08-16: 2503.874156656103
2020-08-17: 2887.7095313749555
2020-08-18: 2504.3675807533805
2020-08-19: 2656.9702178109887
2020-08-20: 2848.757625407416
2020-08-21: 2710.9025006669763
2020-08-22: 2846.5006544979365
2020-08-23: 3036.7270038392003
2020-08-24: 2872.661477561764
2020-08-2

2020-08-10: 3323.4698323151574
2020-08-11: 4589.098396897276
2020-08-12: 3180.220846343904
2020-08-13: 3488.4304582283216
2020-08-14: 3948.428767029156
2020-08-15: 3742.6384109532137
2020-08-16: 3881.6789368988907
2020-08-17: 4434.410612458716
2020-08-18: 3839.2651533677113
2020-08-19: 3982.7591767349872
2020-08-20: 4237.783333123498
2020-08-21: 4156.109242451357
2020-08-22: 4311.960707381879
2020-08-23: 4580.180886478136
2020-08-24: 4328.045488371419
2020-08-25: 4414.981969422375
2020-08-26: 4564.304473372904
2020-08-27: 4555.4486094912845
2020-08-28: 4687.905182610622
2020-08-29: 4841.053814805171
2020-08-30: 4762.439571733516
2020-08-31: 4833.733942658484

Predicting for United States__North Dakota
2020-08-01: 773.5145254434788
2020-08-02: 1025.6460032628088
2020-08-03: 1089.614379695767
2020-08-04: 1118.2012048586903
2020-08-05: 967.0574499716893
2020-08-06: 621.3518464799686
2020-08-07: 972.7514921316838
2020-08-08: 1148.2288939731677
2020-08-09: 1217.797502180208
2020-08-10: 1277

2020-08-24: 1942.1512835592953
2020-08-25: 2013.4583899793283
2020-08-26: 2082.283120701408
2020-08-27: 2103.869677878095
2020-08-28: 2168.168203018515
2020-08-29: 2235.323560927891
2020-08-30: 2247.860821521334
2020-08-31: 2306.6373825822698

Predicting for United States__Pennsylvania
2020-08-01: 5830.427503377539
2020-08-02: 6413.9712428894745
2020-08-03: 6505.130313475134
2020-08-04: 7426.955376748356
2020-08-05: 7397.572675268688
2020-08-06: 3666.9848416004684
2020-08-07: 6064.780703920185
2020-08-08: 6613.497582656129
2020-08-09: 6719.211234283564
2020-08-10: 7498.51977814207
2020-08-11: 7263.147597850673
2020-08-12: 5615.310294263154
2020-08-13: 6635.176064034724
2020-08-14: 7007.934105872908
2020-08-15: 7150.510405490171
2020-08-16: 7690.583856272931
2020-08-17: 7533.67157066698
2020-08-18: 6808.6139808148
2020-08-19: 7269.75917670892
2020-08-20: 7511.857893043372
2020-08-21: 7670.988637116254
2020-08-22: 8034.146175361258
2020-08-23: 7977.419087678205
2020-08-24: 7663.798877621

2020-08-25: 878.3595487277153
2020-08-26: 911.9393178628122
2020-08-27: 939.6043033603843
2020-08-28: 973.7152568736949
2020-08-29: 1004.3768208420654
2020-08-30: 1022.7067254413435
2020-08-31: 1054.8932209395111

Predicting for United States__Washington
2020-08-01: 4171.316193944767
2020-08-02: 3628.7639904632997
2020-08-03: 2761.192866937978
2020-08-04: 1933.8165435254634
2020-08-05: 3351.6343785377603
2020-08-06: 1782.8396184295282
2020-08-07: 3592.296579541612
2020-08-08: 3614.8246596240424
2020-08-09: 3053.898793611572
2020-08-10: 2845.5581361371515
2020-08-11: 3310.899242921557
2020-08-12: 2738.0781002551125
2020-08-13: 3580.754219105605
2020-08-14: 3710.1136473034753
2020-08-15: 3420.3409088572866
2020-08-16: 3395.640628168669
2020-08-17: 3554.920647194553
2020-08-18: 3355.7562019378806
2020-08-19: 3779.9569220269327
2020-08-20: 3905.826495704089
2020-08-21: 3792.3519088836665
2020-08-22: 3824.8011003792803
2020-08-23: 3892.324514414434
2020-08-24: 3825.100901786743
2020-08-25: 

2020-08-30: 936.0402374722796
2020-08-31: 966.5285079774995

Predicting for Yemen__nan
2020-08-01: 130.26612228820377
2020-08-02: 157.53477960596615
2020-08-03: 172.52304031285374
2020-08-04: 191.4744753944079
2020-08-05: 200.79432325439237
2020-08-06: 219.16942577537029
2020-08-07: 291.66619698423665
2020-08-08: 326.0654303373911
2020-08-09: 351.2524643828342
2020-08-10: 377.59781974629516
2020-08-11: 393.3396180969726
2020-08-12: 419.7276912482721
2020-08-13: 471.52650393082047
2020-08-14: 507.6062390241766
2020-08-15: 520.4180318244672
2020-08-16: 546.8233694830503
2020-08-17: 568.5884708480803
2020-08-18: 585.8527113150399
2020-08-19: 625.4666277066528
2020-08-20: 658.1225901395773
2020-08-21: 680.0001242998385
2020-08-22: 707.706113816676
2020-08-23: 727.3679432842396
2020-08-24: 709.0176822791964
2020-08-25: 733.9589201360195
2020-08-26: 759.6944285362947
2020-08-27: 779.8901712937165
2020-08-28: 805.6870055341956
2020-08-29: 823.5351727399606
2020-08-30: 824.830059220488
2020-08

In [18]:
# Check the predictions
preds_df[preds_df.GeoID == 'Italy_nan']

AttributeError: 'DataFrame' object has no attribute 'GeoID'

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [52]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../covid_xprize/validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [53]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,142.56629738919912
Aruba,,2020-08-02,161.10804722795518
Aruba,,2020-08-03,174.4263128199457
Aruba,,2020-08-04,197.16095044264176
Afghanistan,,2020-08-01,266.70666571128135
Afghanistan,,2020-08-02,361.78323642335
Afghanistan,,2020-08-03,373.63396107727465
Afghanistan,,2020-08-04,379.39879996232133
Angola,,2020-08-01,317.3563234888792


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [19]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [22]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../covid_xprize/validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

validate(start_date="2020-08-01",
         end_date="2020-08-30",
         ip_file="../../covid_xprize/validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/try_1_month.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Anguilla__nan  is missing
Cayman Islands__nan  is missing
Falkland Islands__nan  is missing
Gibraltar__nan  is missing
Montserrat__nan  is missing
Pitcairn Islands__nan  is missing
Turks and Caicos Islands__nan  is missing
British Virgin Islands__nan  is missing
Saved predictions to predictions/val_4_days.csv
Done!
Missing countries / regions: {'Anguilla', 'Cayman Islands', 'Falkland Islands', 'Pitcairn Islands', 'Turks and Caicos Islands', 'British Virgin Islands', 'Montserrat', 'Gibraltar'}
Generating predictions from 2020-08-01 to 2020-08-30...
Anguilla__nan  is missing
Cayman Islands__nan  is missing
Falkland Islands__nan  is missing
Gibraltar__nan  is missing
Montserrat__nan  is missing
Pitcairn Islands__nan  is missing
Turks and Caicos Islands__nan  is missing
British Virgin Islands__nan  is missing
Saved predictions to predictions/try_1_month.csv
Done!
Missing countries / regions: {'Anguilla', 'Cayman Islands', 'Falkland Is

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [21]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../covid_xprize/validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Traceback (most recent call last):
  File "predict.py", line 207, in <module>
    predict(args.start_date, args.end_date, args.ip_file, args.output_file)
  File "predict.py", line 52, in predict
    preds_df = predict_df(start_date, end_date, path_to_ips_file, verbose=False)
  File "predict.py", line 175, in predict_df
    pred_df = pd.concat(geo_pred_dfs)
  File "/home/mattia/.local/lib/python3.8/site-packages/pandas-1.1.2-py3.8-linux-x86_64.egg/pandas/core/reshape/concat.py", line 274, in concat
    op = _Concatenator(
  File "/home/mattia/.local/lib/python3.8/site-packages/pandas-1.1.2-py3.8-linux-x86_64.egg/pandas/core/reshape/concat.py", line 331, in __init__
    raise ValueError("No objects to concatenate")
ValueError: No objects to concatenate


FileNotFoundError: [Errno 2] No such file or directory: 'predictions/val_1_month_future.csv'

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [59]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-04
End date: 2021-06-02


In [60]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [61]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-04 to 2021-06-02...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 6.41 s, sys: 859 ms, total: 7.27 s
Wall time: 2min 39s
